In [8]:
!pip install openpyxl


In [40]:
import pandas as pd

def load_and_merge_year(year, patients, svi_path, crosswalk_path):
    import pandas as pd

    # Load ZIP-to-TRACT crosswalk
    crosswalk = pd.read_excel(crosswalk_path, dtype={'ZIP': str, 'TRACT': str})

    # Normalize all column names to uppercase
    crosswalk.columns = crosswalk.columns.str.upper()
    print(crosswalk.columns)  # Optional debug step

    # Ensure ZIP and TRACT are strings
    crosswalk["ZIP"] = crosswalk["ZIP"].astype(str)
    crosswalk["TRACT"] = crosswalk["TRACT"].astype(str)

    # Sort and deduplicate
    crosswalk = crosswalk.sort_values(by=["ZIP", "RES_RATIO"], ascending=[True, False])
    zip_to_fips = crosswalk.drop_duplicates(subset="ZIP", keep="first")[["ZIP", "TRACT"]]
    zip_to_fips.rename(columns={"ZIP": "zip_code", "TRACT": f"FIPS_{year}"}, inplace=True)

    # Ensure patient's zip_code is also string
    patients["zip_code"] = patients["zip_code"].astype(str)

    # Merge ZIP → FIPS_YEAR
    df = patients.merge(zip_to_fips, on="zip_code", how="left")

    # Load SVI_YEAR and merge RPL_THEMES_YEAR
    svi = pd.read_csv(svi_path, dtype={"FIPS": str})
    svi = svi[["FIPS", "RPL_THEMES"]].rename(
        columns={"FIPS": f"FIPS_{year}", "RPL_THEMES": f"RPL_THEMES_{year}"}
    )

    # Merge on FIPS_YEAR
    df = df.merge(svi, on=f"FIPS_{year}", how="left")

    return df


In [42]:
# Load base patient dataset
patients = pd.read_csv("/Users/xuanchen99/Desktop/genAI/project/ZIP___Urban_Rural_Medical_Adherence_Dataset__2024_.csv")
patients["zip_code"] = patients["zip_code"].astype(str)


In [44]:

# Merge 2018
patients_2018 = load_and_merge_year(
    2018, patients,
    svi_path="/Users/xuanchen99/Desktop/genAI/project/SVI_2018_US.csv",
    crosswalk_path="/Users/xuanchen99/Desktop/genAI/project/zip_tract_122018.xlsx"
)

# Merge 2020
patients_2020 = load_and_merge_year(
    2020, patients_2018,
    svi_path="/Users/xuanchen99/Desktop/genAI/project/SVI_2020_US.csv",
    crosswalk_path="/Users/xuanchen99/Desktop/genAI/project/zip_tract_122020.xlsx"
)

# Merge 2022
final_df = load_and_merge_year(
    2022, patients_2020,
    svi_path="/Users/xuanchen99/Desktop/genAI/project/SVI_2022_US.csv",
    crosswalk_path="/Users/xuanchen99/Desktop/genAI/project/zip_tract_122022.xlsx"
)


Index(['ZIP', 'TRACT', 'RES_RATIO', 'BUS_RATIO', 'OTH_RATIO', 'TOT_RATIO'], dtype='object')
Index(['ZIP', 'TRACT', 'RES_RATIO', 'BUS_RATIO', 'OTH_RATIO', 'TOT_RATIO'], dtype='object')


/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Index(['ZIP', 'TRACT', 'USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE',
       'RES_RATIO', 'BUS_RATIO', 'OTH_RATIO', 'TOT_RATIO'],
      dtype='object')


In [46]:
# Save final result
final_df.to_csv("/Users/xuanchen99/Desktop/genAI/project/patients_with_svi_2018_2020_2022.csv", index=False)


In [56]:
# Load the dataset
df = pd.read_csv("/Users/xuanchen99/Desktop/genAI/project/patients_with_svi_2018_2020_2022.csv")

# Define categorization function
def label_svi_change(delta):
    if pd.isna(delta):
        return "Unknown"
    elif delta <= -0.05:
        return "Improved"
    elif delta >= 0.05:
        return "Declined"
    else:
        return "Stable"

# Compute SVI changes
df["SVI_Change_2018_2020"] = df["RPL_THEMES_2020"] - df["RPL_THEMES_2018"]
df["SVI_Change_2020_2022"] = df["RPL_THEMES_2022"] - df["RPL_THEMES_2020"]
df["SVI_Change_2018_2022"] = df["RPL_THEMES_2022"] - df["RPL_THEMES_2018"]

# Assign categories
df["SVI_Change_Category_2018_2020"] = df["SVI_Change_2018_2020"].apply(label_svi_change)
df["SVI_Change_Category_2020_2022"] = df["SVI_Change_2020_2022"].apply(label_svi_change)
df["SVI_Change_Category_2018_2022"] = df["SVI_Change_2018_2022"].apply(label_svi_change)




In [58]:
# Save the new file
df.to_csv("patients_with_three_svi_changes.csv", index=False)